# Préambule : nos biais inconscients

Nous vous proposons, si vous le souhaitez, de prendre une dizaine de minutes pour tester vos biais inconscients:

https://implicit.harvard.edu/implicit/canadafr/takeatest.html


# TD 2: Manipulation des données

## Objectives


 1. Study the data, the distribution of each feature and its relation to the target.

 2. Highlight some bias present in the data


## Installation of the environnement

We highly recommend you to follow these steps, it will allow every student to work in an environment as similar as possible to the one used during testing.

### Colab Settings
  The next two cells of code are too execute only once per colab environment


#### 1. Python env creation

        ```
        ! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```

#### 2. Download MEPS dataset (for part2) it can take several minutes

        ```
        ! Rscript /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/generate_data.R
        ! mv h181.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ! mv h192.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ```

  
### Local Settings

You can use the same env as TD1, you will then need to follow steps 2 and 4

#### 1. Uv installation


        https://docs.astral.sh/uv/getting-started/installation/


        `curl -LsSf https://astral.sh/uv/install.sh | sh`

        Python version 3.12 installation (highly recommended)
        `uv python install 3.12`

#### 2. R installation *NEW*

        In the command `Rscript` says 'command not found'

        `sudo apt install r-base-core`

#### 3. Python env creation

        ```
        mkdir TD_bias_mitigation
        cd TD_bias_mitigation
        uv python pin 3.12
        uv init
        uv pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```

#### 4. Download MEPS dataset, it can take several minutes *NEW*

        ```
        cd TD_bias_mitigation/.venv/lib/python3.12/site-packages/aif360/data/raw/meps/
        Rscript generate_data.R
        ```

## Dataset: Meps 

We recommend consulting the following pages for a better understanding of the dataset: [MEPSDataset19](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.MEPSDataset19.html) and the [AIF360 tutorial](https://github.com/Trusted-AI/AIF360/blob/main/examples/tutorial_medical_expenditure.ipynb)

What you need to have read
- **The sensitive attribute is 'RACE' :1 is privileged, 0 is unprivileged** ; It is constructed as follows: 'Whites' (privileged class) defined by the features RACEV2X = 1 (White) and HISPANX = 2 (non Hispanic); 'Non-Whites' that included everyone else.
(The features 'RACEV2X', 'HISPANX' etc are removed, and replaced by the 'RACE')
- **'UTILIZATION' is the outcome (the label to predict for a ML model) 0 is positive 1 is negative**. It is a binary composite feature, created to measure the total number of trips requiring some sort of medical care, it sum up the following features (that are removed from the data):
    * OBTOTV15(16), the number of office based visits
    * OPTOTV15(16), the number of outpatient visits
    * ERTOT15(16), the number of ER visits
    * IPNGTD15(16), the number of inpatient nights
    * HHTOTD16, the number of home health visits
UTILISATION is set to 1 when te sum is above or equal to 10, else it is set to 0
- **The dataset is weighted** The dataset come with an 'instance_weights' attribute that corresponds to the feature perwt15f these weights are supposed to generate estimates that are representative of the United State (US) population in 2015.


Summary to remember
- **The sensitive attribute is 'RACE' :1 is privileged, 0 is unprivileged**
- **'UTILIZATION' is the outcome (the label to predict for a ML model) 0 is positive 1 is negative**
- **The dataset is weighted**


In [1]:
# To execute only in Colab
! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit

/home/cours_fairness/L3_2026/TD_corr/.venv/bin/python: No module named pip


In [2]:
# Code to compute fairness metrics using aif360

from aif360.sklearn.metrics import *
from sklearn.metrics import  balanced_accuracy_score

 
# This method takes lists
def get_metrics(
    y_true, # list or np.array of truth values
    y_pred=None,  # list or np.array of predictions
    prot_attr=None, # list or np.array of protected/sensitive attribute values
    priv_group=1, # value taken by the privileged group
    pos_label=1, # value taken by the positive truth/prediction
    sample_weight=None # list or np.array of weights value,
):
    group_metrics = {}
    group_metrics["base_rate_truth"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["base_rate_preds"] = base_rate(
        y_true=y_pred, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        if len(set(y_pred))>1:
            group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
                y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
            )
        else:
            group_metrics["conditional_demographic_disparity"] =None
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["balanced_accuracy_score"] = balanced_accuracy_score(
        y_true=y_true, y_pred=y_pred, sample_weight=sample_weight
        )
    return group_metrics

2025-12-15 16:57:28.583506: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/cours_fairness/L3_2026/TD_corr/.venv/lib/python3.12/site-packages/aif360/sklearn/inprocessing/infairness.py:265: SyntaxWarning: invalid escape sequence '\e'
  eps (float): :math:`\epsilon` parameter in the SenSeI algorithm.
/home/cours_fairness/L3_2026/TD_corr/.venv/lib/python3.12/site-packages/aif360/sklearn/inprocessing/infairness.py:344: SyntaxWarning: invalid escape sequence '\e'
  eps (float): :math:`\epsilon` parameter in the SenSR algorithm.
/home/cours_fairness/L3_2026/TD_corr/.venv/lib/python3.12/site-packages/inFairness/fairalgo/sensei.py:27: SyntaxWarning: invalid escape sequence '\e'
  :math:`\epsilon` parameter in the SenSR algorithm
/home/cours_fairness/L3_202

In this TD we will use data from the [Medical Expenditure Panel Survey](https://meps.ahrq.gov/mepsweb/). The TD is inspired from [AIF360 tutorial](https://github.com/Trusted-AI/AIF360/blob/main/examples/tutorial_medical_expenditure.ipynb)

## The Meps dataset

In [3]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', append=True, category=UserWarning)


In [4]:
# Datasets
from aif360.datasets import MEPSDataset19
from aif360.datasets import MEPSDataset20
from aif360.datasets import MEPSDataset21

MEPSDataset19_data = MEPSDataset19()
# (dataset_orig_panel19_train,
#  dataset_orig_panel19_val,
#  dataset_orig_panel19_test) = MEPSDataset19_data.split([0.5, 0.8], shuffle=True)

In [5]:
(dataset_orig_panel19_train,
 dataset_orig_panel19_val,
 dataset_orig_panel19_test) = MEPSDataset19().split([0.5, 0.8], shuffle=True)

In [6]:
len(dataset_orig_panel19_train.instance_weights), len(dataset_orig_panel19_val.instance_weights), len(dataset_orig_panel19_test.instance_weights)

(7915, 4749, 3166)

In [7]:
instance_weights = MEPSDataset19_data.instance_weights
instance_weights


array([21854.981705, 18169.604822, 17191.832515, ...,  3896.116219,
        4883.851005,  6630.588948], shape=(15830,))

In [8]:
f"Taille du dataset {len(instance_weights)}, poids total du dataset {instance_weights.sum()}."

'Taille du dataset 15830, poids total du dataset 141367240.546316.'

### Premier appercu du dataset

La librairie AIF360 fournie une surcouche au dataset, cela le rend un peu moins intuitif d'utilisation (par exemple pour étudier/visualiser les attributs un à un), mais elle permet de calculer les métrique des fairness en une ligne de commande.

In [9]:
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

metric_orig_panel19_train = BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{'RACE': 0}],
        privileged_groups=[{'RACE': 1}])

print(metric_orig_panel19_train.disparate_impact())

0.49826823461176517


Cependant le but de ce TD étant encore de manipuler les données et de les analyser nous allons revenir aux données sous forme d'un dataframe.

Note pour calculer les métriques de fairness sans avoir à les réimplémenter dans le cas pondéré (instances weights) vous pouvez utiliser les méthodes implémenter dans AIF360 là [Implémentation Métriques de Fairness](https://aif360.readthedocs.io/en/latest/modules/sklearn.html#module-aif360.sklearn.metrics)

### Conversion en un dataframe

Nous avons vu que la somme des poids est conséquente, pres de 115millions nous ne pouvons donc pas raisonneblement dupliqué chaque ligne autant de fois que son poids.

Nous allons stocker la pondération et la prendre en compte ensuite dans notre analyse

In [10]:
def get_df(MepsDataset):
    data = MepsDataset.convert_to_dataframe()
    # data_train est un tuple, avec le data_frame et un dictionnaire avec toutes les infos (poids, attributs sensibles etc)
    df = data[0]
    df['WEIGHT'] = data[1]['instance_weights']
    return df

df = get_df(MEPSDataset19_data)



In [11]:
from aif360.sklearn.metrics import disparate_impact_ratio, base_rate
dir = disparate_impact_ratio(
    y_true=df.UTILIZATION, 
    prot_attr=df.RACE, 
    pos_label=0,
    sample_weight=df.WEIGHT)
br =base_rate(
    y_true=df.UTILIZATION, 
    pos_label=0,
    sample_weight=df.WEIGHT)
dir,br

(1.1848351529675123, np.float64(0.7849286063696154))

In [12]:
dir = disparate_impact_ratio(
    y_true=df.UTILIZATION, 
    prot_attr=df.RACE, 
    pos_label=0)
br =base_rate(
    y_true=df.UTILIZATION, 
    pos_label=0)
dir,br

(1.1746792888264614, np.float64(0.8283006948831333))

## Question 1 - Apprendre un modèle pour prédire le fait d'être réadmis
### 1.1 - Faire le pre-processing des données

Ici ce pre-processing a déjà été fait par AIF, nous avons simplement converti le dataset en dataframe pour pouvoir le manipuler librement

### Question 1.2 - Creer les échantillons d'apprentissage, de validation et de test

Pour créer le df_X il faut enlever l'outcome ("UTILIZATION") et la pondération ("WEIGHT")

La colonne "UTILIZATION" sera le label (noté y)

La colonne "WEIGHT" sera la pondération (notée w)


In [44]:
from sklearn.model_selection import train_test_split
df_X = df.drop(columns=["UTILIZATION", "WEIGHT"])
splits_trainval_test = train_test_split(
    df_X, df["UTILIZATION"], df["WEIGHT"],
    train_size=0.8, 
    random_state=42)

X_trainval, X_test, y_trainval, y_test, w_trainval, w_test = splits_trainval_test
splits_train_val = train_test_split(
    X_trainval, y_trainval, w_trainval,
    train_size=0.625, 
    random_state=42)
X_train, X_val, y_train, y_val, w_train, w_val = splits_train_val

In [45]:
df_X.shape, X_train.shape, y_train.shape, w_train.shape, X_val.shape, y_val.shape, w_val.shape,  X_test.shape, y_test.shape, w_test.shape

((15830, 138),
 (7915, 138),
 (7915,),
 (7915,),
 (4749, 138),
 (4749,),
 (4749,),
 (3166, 138),
 (3166,),
 (3166,))

### Question 1.3 - Apprendre une regression logistique dont le but est de prédire UTILIZATION

In [46]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline


model = make_pipeline(StandardScaler(),LogisticRegression(random_state=42))

model = model.fit(X_train, y_train, **{'logisticregression__sample_weight':w_train})

preds = model.predict(X_val)

model.score(X_val, y_val, sample_weight=w_val)

0.8429447781148914

### Quesiton 1.4 Performance du modèle (afficher la matrice de confusion)

In [47]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_val, preds).ravel()
print(tn, fp, fn, tp)
fig = px.imshow([[tn, fp], [fn, tp]], text_auto=True, labels=dict(y="Truth", x="Pred"),
                x=["False", "True"],
                y=["False", "True"]
               )
fig.show()

3773 176 456 344


### Question 1.5 - Calculer les métriques "group fairness" du modèle

In [ ]:
get_metrics(
    y_true=y_val, # list or np.array of truth values
    y_pred=y_val,  # list or np.array of predictions
    prot_attr=X_val["RACE"], # list or np.array of protected/sensitive attribute values
    pos_label=0, # value taken by the positive truth/prediction
    sample_weight=w_val # list or np.array of weights value,
)

{'base_rate_truth': np.float64(0.7824798836724106),
 'statistical_parity_difference': np.float64(0.14593258594320302),
 'disparate_impact_ratio': 1.2014878422901056,
 'base_rate_preds': np.float64(0.7824798836724106),
 'equal_opportunity_difference': 0.0,
 'average_odds_difference': 0.0,
 'conditional_demographic_disparity': np.float64(0.04158857354105518),
 'smoothed_edf': np.float64(0.7534670691209804),
 'df_bias_amplification': np.float64(0.0),
 'balanced_accuracy_score': 1.0}

In [50]:
get_metrics(
    y_true=y_val, # list or np.array of truth values
    y_pred=preds,  # list or np.array of predictions
    prot_attr=X_val["RACE"], # list or np.array of protected/sensitive attribute values
    pos_label=0, # value taken by the positive truth/prediction
    sample_weight=w_val # list or np.array of weights value,
)


{'base_rate_truth': np.float64(0.7824798836724106),
 'statistical_parity_difference': np.float64(0.1293902930520614),
 'disparate_impact_ratio': 1.1607532512503969,
 'base_rate_preds': np.float64(0.8565071268793254),
 'equal_opportunity_difference': 0.045029230285676736,
 'average_odds_difference': 0.12112029660569726,
 'conditional_demographic_disparity': np.float64(0.05106634071491531),
 'smoothed_edf': np.float64(1.0882653239012994),
 'df_bias_amplification': np.float64(0.334798254780319),
 'balanced_accuracy_score': 0.7078853423264241}

## Question 2 - Etude de l'impact de la couleur de peau sur le faire d'être réadmis, les prédictions du modèle et ses liens avec les autres variables


### Question 2.1 - Faire l'étude descriptive univarié de la couleur de peau ('RACE') (effectif, fréquence, model)

In [19]:
interest = "RACE"
# on traite le cas où la variable est categorielle et a été one hot encodée
interest_col = [col for col in df_X.columns if col[:len(interest)]==interest]
def interest_transform(X, interest_col, interest, new_name):
    X_interest = X.drop(columns=interest_col)
    if len(interest_col)>1:
        X_interest[new_name] = df_X[interest_col].apply(
            lambda x: interest_col[np.argmax(x)][len(interest)+1:], axis=1)
    else:
        X_interest[new_name] = df_X[interest_col]
    return X_interest

X_train_interest = interest_transform(X_train, interest_col, interest, new_name="interest")
X_val_interest = interest_transform(X_val, interest_col, interest, new_name="interest")


In [20]:
X_train_interest

,AGE,PCS42,MCS42,K6SUM42,REGION=1,REGION=2,REGION=3,REGION=4,SEX=1,SEX=2,...,EMPST=4,POVCAT=1,POVCAT=2,POVCAT=3,POVCAT=4,POVCAT=5,INSCOV=1,INSCOV=2,INSCOV=3,interest
10862,45.0,48.45,51.25,2.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
11755,79.0,25.38,56.48,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
11708,6.0,-1.00,-1.00,-1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
12782,3.0,-1.00,-1.00,-1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1544,9.0,-1.00,-1.00,-1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4585,2.0,-1.00,-1.00,-1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
11815,60.0,43.99,57.09,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
496,62.0,27.16,29.66,13.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
7753,65.0,36.10,53.48,5.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [21]:
def analyse_univarie(interet, df , weights):
  effectif = {}
  sum_effectif = 0
  max_effectif = 0
  for value in df[interet].unique():
    effectif[value] = weights[df[interet]==value].sum()
    sum_effectif += effectif[value]
    if effectif[value] > max_effectif:
      mode = value
  frequence = {k:v/sum_effectif for k,v in effectif.items()}
  return effectif, frequence, mode

In [22]:
# Analyse pondérée
effectif, frequence, mode = analyse_univarie("interest", df = X_train_interest, weights = w_train)

print("Effectif: ", effectif)
print("Frequence: ", frequence)
print("Mode: ", mode)

Effectif:  {np.float64(1.0): np.float64(41677132.696059), np.float64(0.0): np.float64(28847296.25972)}
Frequence:  {np.float64(1.0): np.float64(0.5909602291454477), np.float64(0.0): np.float64(0.4090397708545524)}
Mode:  0.0


In [23]:
# Analyse non pondérée
no_w_train = pd.Series(data=[1.0 for _ in w_train], index=w_train.index, name=w_train.name)
effectif, frequence, mode = analyse_univarie("interest", df = X_train_interest, weights = no_w_train)

print("Effectif: ", effectif)
print("Frequence: ", frequence)
print("Mode: ", mode)

Effectif:  {np.float64(1.0): np.float64(2754.0), np.float64(0.0): np.float64(5161.0)}
Frequence:  {np.float64(1.0): np.float64(0.3479469361970941), np.float64(0.0): np.float64(0.6520530638029058)}
Mode:  0.0


### Question 2.2 - Faire des graphiques décrivant la couleur de peau  (diagramme en secteur, diagramme en barres)

In [24]:
def graphique_univarie(interet, df, weights):
    effectif, _, _ = analyse_univarie(interet, df = df, weights=weights)

    df_interet = pd.DataFrame.from_dict([{"effectif":k, "count":v} for k,v in effectif.items()])
    # df_interet.rename(columns = {'index':interet, interet:"effectif"},
    #         inplace = True)
    pie = px.pie(df_interet, values='count', names = "effectif")
    bar = px.bar(df_interet, y='count', x="effectif", text_auto=True)

    return pie, bar

pie, bar = graphique_univarie(interet="interest", df = X_train_interest, weights=w_train)

In [25]:
pie

In [26]:
bar

In [27]:
# Non pondéré

pie, bar = graphique_univarie(interet="interest", df = X_train_interest, weights=no_w_train)
pie

In [28]:
bar

### Question 2.3 - Faire l'analyse bivariée entre la couleur de peau et les autres variables explicatives quantitatives (boite à moustaches des variables par genre, densité/histogramme par genre, rapport de corrélation)

In [29]:
import plotly.graph_objects as go


def weighted_hist(interet, numeric, df, weights, histnorm=None):
    fig = go.Figure()

    for i in np.unique(df[interet]):
        idx_i = (df[interet] == i)
        counts, bin_edges = np.histogram(df[numeric].values[idx_i], bins=25, weights=weights.values[idx_i])
        if histnorm=="probability":
            counts*=100/sum(counts)
        fig.add_trace(go.Bar(x=bin_edges, y=counts))
    # Overlay both histograms
    fig.update_layout(barmode='overlay')
    # Reduce opacity to see both histograms
    fig.update_traces(opacity=0.75)
    fig.show()
    return fig

def weighted_box(df, x, y, weights):
    weight_col = "weighted_value"
    df_plot = pd.concat([df[x].copy(), df[y].copy(), weights.copy()], axis=1)
    df_plot["weighted_value"] = df_plot[[y,"WEIGHT"]].apply(lambda x: x[0]*x[1], axis=1)
    df_plot.sort_values(y, inplace=True)
    fig = go.Figure()
    for i in np.unique(df[x]):
        df_i = df_plot[df[x]==i]
        totsum = df_i[weight_col].sum()
        cumsum = df_i[weight_col].cumsum()
        q1 = df_i[y][cumsum >= totsum/4.0].iloc[0]
        q3 = df_i[y][cumsum >= totsum*0.75].iloc[0]
        iqr = q3-q1
        fig.add_trace(go.Box(
            median = [df_i[y][cumsum >= totsum/2.0].iloc[0]],
            q1 = [q1],
            q3 = [q3],
            mean = [(df_i[weight_col] * df_i[y]).sum()/totsum],
            lowerfence = [df_i[y][cumsum <= max(q1 - 1.5*iqr, cumsum[0])].iloc[0]],
            upperfence = [df_i[y][cumsum >= min(q3 + 1.5*iqr, cumsum[-1])].iloc[0]],
            name=f"{x}={i}"
        ))
    fig.update_layout(boxmode='group', xaxis_tickangle=45, title=f"Box plot of {y} by values of {x}")
    
    return fig
    


def variance_explique_une_valeur(interet,value_interet, numeric, mean_numeric, df, weights):
    idx_group = (df[interet] == value_interet)
    df_group = df[idx_group]
    len_group = weights[idx_group].sum()
    mean_group = np.mean([v*w for v,w in zip(df_group[numeric],weights[idx_group])])

    ret = len_group * (mean_group - mean_numeric)* (mean_group - mean_numeric)
    return ret

def variance_explique(interet, numeric, df, weights):
    mean_numeric = np.mean([v*w for v,w in zip(df[numeric],weights)])
    unique_interet = np.unique(df[interet])
    ret = 1/weights.sum() * np.sum([
        variance_explique_une_valeur(interet=interet, value_interet=i, numeric=numeric, mean_numeric=mean_numeric, df= df, weights=weights) 
        for i in unique_interet])
    return ret

def rapport_correlation(interet, numeric, df, weights):

    ret = variance_explique(interet, numeric, df=df, weights=weights)/np.var([v*w for v,w in zip(df[numeric],weights)])
    return ret


def analyse_bivarie_num(interet, numeric, df, weights):
    # bam = px.box(df, x=interet, y=numeric)
    bam = weighted_box(df, x=interet, y=numeric, weights=weights)
    bam.show()
    # histogramme
    fig_1 = weighted_hist(interet=interet, numeric=numeric, df=df, weights=weights)
    fig_2 = weighted_hist(interet=interet, numeric=numeric, df=df, weights=weights, histnorm="probability")
    #hist_1 = px.histogram(df, x=numeric, color = interet, histnorm='probability density')
    #hist_1.show()
    hist = px.histogram(df, y=numeric, x = interet)
    hist.show()

    rp_corr = rapport_correlation(interet=interet, numeric=numeric, df = df, weights=weights)
    print("rapport correlation: ", rp_corr)

    return fig_1,fig_2, hist, bam, rp_corr


def analyse_bivarie_num_all(interet, numeric_feature, df, weights):

    ret = [analyse_bivarie_num(interet=interet, numeric=i, df=df, weights=weights) for i in numeric_feature]
    return ret

numerical_features = [
    col for col 
    in list(X_train_interest.select_dtypes(include=np.number).columns)
    if not(('=' in col) or (col=="interest")) ] # Remove one hot encoded categorial feature
print(numerical_features)
graphic = analyse_bivarie_num_all(interet="interest", numeric_feature = numerical_features, df=X_train_interest, weights=w_train)


['AGE', 'PCS42', 'MCS42', 'K6SUM42']


rapport correlation:  0.3067200610093844


rapport correlation:  0.2160508445361389


rapport correlation:  0.22799678406158752


rapport correlation:  0.06759288918608035


In [30]:
# Non pondérée
graphic = analyse_bivarie_num_all(interet="interest", numeric_feature = numerical_features, df=X_train_interest, weights=no_w_train)

rapport correlation:  0.028646967897857707


rapport correlation:  0.009548514477493301


rapport correlation:  0.009563168007337362


rapport correlation:  0.01078951355521699


### Question 2.4 - Faire l'analyse bivariée entre la couelur de peau et d'autres variables explicatives qualitative (table de contingence, diagramme en barre selon les profils lignes et selon les profils colonnes, diagramme en mosaique)

In [31]:
# il faut d'abord retrouver les variables categorielles qui ont été one_hot_encodées
one_hot_col = [col for col in X_train.columns if "=" in col]
categorial_col = list(set([col.split('=')[0] for col in one_hot_col]))
categorial_col, len(categorial_col), len(one_hot_col)

(['COGLIM',
  'CHBRON',
  'MIDX',
  'JTPAIN',
  'OHRTDX',
  'WLKLIM',
  'EMPST',
  'ARTHDX',
  'CANCERDX',
  'CHOLDX',
  'INSCOV',
  'FTSTU',
  'ACTDTY',
  'ASTHDX',
  'ACTLIM',
  'RTHLTH',
  'PHQ242',
  'EMPHDX',
  'ADHDADDX',
  'STRKDX',
  'HIBPDX',
  'ANGIDX',
  'SOCLIM',
  'POVCAT',
  'DFSEE42',
  'DIABDX',
  'DFHEAR42',
  'ARTHTYPE',
  'ADSMOK42',
  'MARRY',
  'REGION',
  'CHDDX',
  'SEX',
  'HONRDC',
  'PREGNT',
  'MNHLTH'],
 36,
 133)

In [32]:
def analyse_bi_quali_quali(quali1, quali2, df):
  if quali1==quali2:
    return

  # table de contingence
  # on groupe le dataframe sur les variables d'intéret
  df_group = df[[quali1, quali2, "WEIGHT"]].groupby([quali1,quali2]).sum().reset_index()
  df_group.rename(columns={"WEIGHT":"count"}, inplace=True)

  rows = list(set(df_group[quali2].values))
  cols = list(set(df_group[quali1].values))
  contingence_tab = [
      [ df_group[(df_group[quali2]==row) & (df_group[quali1]==col)]["count"].values[0] if col in df_group[df_group[quali2]==row][quali1].values else 0 for row in rows  ]
      for col in cols]
  contingence_img = px.imshow(contingence_tab,
                              text_auto=True,
                              labels=dict(x=quali2, y=quali1),
                              x=rows, y=cols)
  contingence_img.show()

  # diagrammes en barres
  quali1_count = df_group[[quali1, "count"]].groupby([quali1]).sum()
  df_group['count_quali1_prop'] = df_group[[quali1,"count"]].apply(lambda x: 100*x['count']/quali1_count.loc[x[quali1]], axis=1)
  bar1 = px.bar(df_group, x=quali1, y="count_quali1_prop", color=quali2, text_auto=True)
  bar1.show()

  quali2_count = df_group[[quali2, "count"]].groupby([quali2]).sum()
  df_group['count_quali2_prop'] = df_group[[quali2,"count"]].apply(lambda x: 100*x['count']/quali2_count.loc[x[quali2]], axis=1)
  bar2 = px.bar(df_group, x=quali2, y="count_quali2_prop", color=quali1, text_auto=True)
  bar2.show()

In [33]:
cat_col_study = ["MARRY", "PREGNT", "CANCERDX"]

for cat in cat_col_study:
    interest_col = [col for col in X_train_interest.columns if col.split('=')[0]==cat]
    X_study = interest_transform(X_train_interest, interest_col=interest_col, interest=cat, new_name=cat)
    X_study["WEIGHT"] = w_train
    print(cat, X_train.shape, X_study.shape)
    analyse_bi_quali_quali("interest", cat, X_study)

MARRY (7915, 138) (7915, 130)


PREGNT (7915, 138) (7915, 137)


CANCERDX (7915, 138) (7915, 137)


In [34]:
cat="MARRY"
interest_col = [col for col in X_train_interest.columns if col.split('=')[0]==cat]
X_study = interest_transform(X_train_interest, interest_col=interest_col, interest=cat, new_name=cat)
X_study.MARRY.value_counts()

MARRY
1     2627
5     2142
6     1964
3      660
2      295
4      162
7       42
9       11
10      10
8        2
Name: count, dtype: int64

In [35]:
X_study.interest.value_counts()

interest
0.0    5161
1.0    2754
Name: count, dtype: int64

In [36]:
# Non pondérée
cat_col_study = ["MARRY", "PREGNT", "CANCERDX"]

for cat in cat_col_study:
    interest_col = [col for col in X_train_interest.columns if col.split('=')[0]==cat]
    X_study = interest_transform(X_train_interest, interest_col=interest_col, interest=cat, new_name=cat)
    X_study["WEIGHT"] = no_w_train
    print(cat, X_train.shape, X_study.shape)
    analyse_bi_quali_quali("interest", cat, X_study)

MARRY (7915, 138) (7915, 130)


PREGNT (7915, 138) (7915, 137)


CANCERDX (7915, 138) (7915, 137)


### Question 2.5 - Faire l'analyse bivariée entre la couleur de peau et la colonne 'UTILIZATION'

In [37]:
X_study = X_train_interest.copy()
X_study["WEIGHT"] = w_train
X_study["UTILIZATION"] = y_train
analyse_bi_quali_quali("interest","UTILIZATION",X_study)

### Question 2.6 - Faire l'analyse bivariée entre la couleur de peau et les prédictions du modèle prédisant la colonne 'UTILIZATION'

In [38]:
X_study["PREDS"] = model.predict(X_train)
analyse_bi_quali_quali("interest","PREDS",X_study)

### Question 2.7 - Faire l'analyse bivariée entre la couleur de peau et les erreurs du modèle précédent

In [39]:
X_study["ERRORS"] = (X_study["PREDS"]==X_study["UTILIZATION"])
analyse_bi_quali_quali("interest","ERRORS",X_study)


### Question 2.8 - Proposer un modèle à base d'une foret aléatoire prédisant la couleur de peau en fonction des autres variables explicatives

In [40]:
rf_model = make_pipeline(StandardScaler(),RandomForestClassifier(random_state=42))

y_train_interest_rf = X_train_interest["interest"]
X_train_interest_rf = X_train_interest.drop(columns="interest")
y_val_interest_rf = X_val_interest["interest"]
X_val_interest_rf = X_val_interest.drop(columns="interest")

rf_model = rf_model.fit(X_train_interest_rf, y_train_interest_rf, **{'randomforestclassifier__sample_weight':w_train})

rf_preds = rf_model.predict(X_val_interest_rf)

rf_model.score(X_val_interest_rf, y_val_interest_rf, sample_weight=w_val)

0.5907757020105011

### Question 2.9 - Apprendre un modèle privé de la couleur de peau pour prédire UTILIZATION et étudier le lien entre ses prédictions et la couleur de peau

In [41]:
lr_model = make_pipeline(StandardScaler(),LogisticRegression(random_state=42))

lr_model = lr_model.fit(X_train_interest_rf, y_train, **{'logisticregression__sample_weight':w_train})

lr_preds = lr_model.predict(X_val_interest_rf)

lr_model.score(X_val_interest_rf, y_val, sample_weight=w_val)

0.8380014724909729

In [ ]:
lr_model_group_metrics = get_metrics( 
    y_true = y_val,
    y_pred=lr_preds,
    prot_attr= X_val["RACE"], 
    pos_label=0,
    sample_weight=w_val
    )
lr_model_group_metrics

{'statistical_parity_difference': np.float64(0.10833479052325923),
 'disparate_impact_ratio': 1.132451409650838,
 'equal_opportunity_difference': 0.035844858471375196,
 'average_odds_difference': 0.0848643158560019,
 'conditional_demographic_disparity': np.float64(0.043942585046545406),
 'smoothed_edf': np.float64(0.9038379725136674),
 'df_bias_amplification': np.float64(0.150370903392687)}

In [43]:
model_group_metrics

{'statistical_parity_difference': np.float64(0.1293902930520614),
 'disparate_impact_ratio': 1.1607532512503969,
 'equal_opportunity_difference': 0.045029230285676736,
 'average_odds_difference': 0.12112029660569726,
 'conditional_demographic_disparity': np.float64(0.05106634071491531),
 'smoothed_edf': np.float64(1.0882653239012994),
 'df_bias_amplification': np.float64(0.334798254780319)}

## Question 3 - Faire de meme pour une autre variable sensible (Age, genre, marry etc)


## Question 4 - Causalité: appliquer le module causal-learn sur le jeu de données en testant plusieurs des méthodes implémentées et en comparant les résultats entre les méthodes